# _Transformation Or Localisations_

In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

StatementMeta(, 221c552c-7980-4aca-ba34-2ace8a1241f3, 3, Finished, Available, Finished)

In [2]:
dfLocation=spark.read.parquet("Files/Argent/TransformLocation.parquet")
dfLocation=dfLocation.distinct()
for column in dfLocation.columns:
    if column in ["date","created","updated"]:
        dfLocation=dfLocation.withColumn(column, date_format(column,"dd/MM/yyyy"))

dfLocation=dfLocation[dfLocation["osm_address_country"]=="France"]

StatementMeta(, 221c552c-7980-4aca-ba34-2ace8a1241f3, 4, Finished, Available, Finished)

In [ ]:
dfLocation.write.format("parquet").mode("overwrite").save("Files/Or/Locations.parquet")
dfLocation.write.format("delta").mode("overwrite").saveAsTable("LocationTable")

# _Transformation Or Prix_

In [5]:
dfPrice=spark.read.parquet("Files/Argent/TransformPrices.parquet")
dfPrice=dfPrice.distinct()
for column in dfPrice.columns:
    if column in ["date","created","updated"]:
        dfPrice=dfPrice.withColumn(column, date_format(column,"dd/MM/yyyy"))

dfPrice=dfPrice[dfPrice["currency"]=="EUR"]
dfPrice=dfPrice.join(dfLocation,dfLocation["id"]==dfPrice["id"],"inner").select(dfPrice["*"])
dfPrice=dfPrice[(dfPrice["product_id"]!="")&(dfPrice["product_id"].isNotNull())]

StatementMeta(, 221c552c-7980-4aca-ba34-2ace8a1241f3, 7, Finished, Available, Finished)

In [ ]:
dfPrice.write.format("parquet").mode("overwrite").save("Files/Or/Prices.parquet")
dfPrice.write.format("delta").mode("overwrite").saveAsTable("PriceTable")

# _Transformation Or Produits_

In [ ]:
dfProduct=spark.read.parquet("Files/Argent/Transformproducts.parquet")
dfProduct=dfProduct.distinct()
dfProduct=dfProduct.filter((dfProduct.product_name.isNotNull())&(dfProduct.product_name!=""))
for column in dfProduct.columns:
    if column in ["date","created","updated"]:
        dfProduct=dfProduct.withColumn(column, date_format(column,"dd/MM/yyyy"))


dfProduct=dfProduct.join(dfPrice,dfProduct["id"]==dfPrice["product_id"],"inner").select(dfProduct["*"])
dfProduct=dfProduct.distinct()

In [ ]:
dfProduct.write.format("parquet").mode("overwrite").save("Files/Or/Products.parquet")
dfProduct.write.format("delta").mode("overwrite").saveAsTable("ProductTable")